In [72]:
## Réécriture en Julia du code top99 de OLE SIGMUND
using SparseArrays
# que 2 ddl considérés : x et y
# propriétés du matériau 
E = 1.; 
nu = 0.3;
k=[ 1/2-nu/6   1/8+nu/8 -1/4-nu/12 -1/8+3*nu/8 -1/4+nu/12 -1/8-nu/8  nu/6  1/8-3*nu/8];
#matrice de rigidité élémentaire pour 1 élément
KE = E/(1-nu^2)*[ k[1] k[2] k[3] k[4] k[5] k[6] k[7] k[8];
                  k[2] k[1] k[8] k[7] k[6] k[5] k[4] k[3];
                  k[3] k[8] k[1] k[6] k[7] k[4] k[5] k[2];
                  k[4] k[7] k[6] k[1] k[8] k[3] k[2] k[5];
                  k[5] k[6] k[7] k[8] k[1] k[2] k[3] k[4];
                  k[6] k[5] k[4] k[3] k[2] k[1] k[8] k[7];
                  k[7] k[4] k[5] k[2] k[3] k[8] k[1] k[6];
                  k[8] k[3] k[2] k[5] k[4] k[7] k[6] k[1]];


In [73]:
function top(nelx,nely,volfrac,penal,rmin)
    
    # Effort appliqué. F[2,1] = -1 -> sur le ddl 2 (y) du premier pixel (en haut à gauche ) on applique un effort de -1 ( donc vers le haut)
    ddl = 2*(nely+1)*(nelx+1)
    #F = spzeros(ddl);
    F= zeros(ddl);
    F[2] = -1; # modifier pour s'amuser plus tard
    
    #noeuds fixés et libres ( ici moitié de poutre MBB)    
    fixeddofs   = union(1:2:2*(nely+1),[ddl]); #noeuds fixes, on fixe artificiellement x des noeuds à gauche pour la symétrie, on fixe y pour le noeud en bas à droite
    alldofs     = 1:ddl;
    freedofs    = setdiff(alldofs,fixeddofs);
    
    x = volfrac * ones(nely,nelx) ; #répartition uniforme de la masse
    loop = 0; 
    change = 1.; #arbitraire, pour commencer la boucle
    dc = zeros(nely,nelx);
    while change > 0.01  
        loop += 1;
        xold = x; # x old est l'ancienne répartition de la matière        
        c = 0.; # fonction objectif : Edéformation
        U = FE(nelx,nely,x,penal, freedofs, fixeddofs,F);
        for ely = 1:nely
            for elx = 1:nelx
                n1 = (nely+1)*(elx-1)+ely; 
                n2 = (nely+1)* elx   +ely;
                Ue = append!([U[2*n1-1]],append!([U[2*n1]],append!([U[2*n2-1]],append!([U[2*n2]],append!([U[2*n2+1]],append!([U[2*n2+2]],append!([U[2*n1+1]],[U[2*n1+2]]))))))); #les 4 noeuds de l'élem selon x et y
                c = c + x[ely,elx]^penal*Ue'*KE*Ue; # participation élémentaire à Edéf 
                dc[ely,elx] = -penal*x[ely,elx]^(penal-1)*Ue'*KE*Ue; # dc/dx (Xe)
            end
        end
        dc = check(nelx,nely,rmin,x,dc);
        x  = OC(nelx,nely,x,volfrac,dc);
        change = maximum(abs.(x-xold));
        println("change = ", change);
        if loop==50
            return x
        end
    end
    return x
end

top (generic function with 1 method)

In [74]:
function OC(nelx,nely,x,volfrac,dc)  # nouvelle répartition de masse
    l1 = 0; l2 = 100000; move = 0.2;
    xnew = zeros(nely,nelx);    
    while (l2-l1 > 1e-4) #recherche par dichotomie du multiplicateur de Lagrange
        lmid = 0.5*(l2+l1);
        RacBe = sqrt.(-dc/lmid) ; # racine de Be = -dc/(l * dv) et Xe* Be^1/2# xnew € [x-move, x+move]
        XB = x.*RacBe ;
        for i = 1:nelx
            for j = 1:nely
                xji = x[j,i];
                xnew[j,i]= max(0.001,max(xji-move,min(1,min(xji+move,XB[j,i]))));
            end
        end        
        if sum(sum(xnew)) - volfrac*nelx*nely > 0;
            l1 = lmid;
        else
            l2 = lmid;
        end
    end
    return xnew
end

OC (generic function with 1 method)

In [75]:
function FE(nelx,nely,x,penal, freedofs, fixeddofs, F) #Création de la matrice de rigidité globale de la structure, Résolution des déplacements
    K = spzeros(2*(nelx+1)*(nely+1), 2*(nelx+1)*(nely+1)); # matrice creuse de rigidité globale, le x2 car 2 ddl    
    U = zeros(2*(nely+1)*(nelx+1));
    for elx = 1:nelx
      for ely = 1:nely #boucle sur tous les éléments
        n1 = (nely+1)*(elx-1)+ely; #noeud en haut à gauche de l'elem 
        n2 = (nely+1)* elx   +ely; #en haut à droite
        edof = [2*n1-1; 2*n1; 2*n2-1; 2*n2; 2*n2+1; 2*n2+2; 2*n1+1; 2*n1+2]; # les 2 ddl des 4 coins de l'élem
        K[edof,edof] += x[ely,elx]^penal*KE; # c(x) = somme sur elem e( Xe^p * Ue' * KE * Ue), d'où K = (Xe^p*KE)e
      end
    end
    U[freedofs] = K[freedofs,freedofs] \ F[freedofs]; 
    U[fixeddofs] .= 0;
    return U
end

FE (generic function with 2 methods)

In [63]:
function check(nelx,nely,rmin,x,dc) # filtrage assurant l'existance d'une solution
    dcn=zeros(nely,nelx); #dc/dx filtrée. Hf = Rmin - dist(e,f)
    for i = 1:nelx
      for j = 1:nely
        sum=0.0;
        for k = max(i-floor(rmin),1):min(i+floor(rmin),nelx)
          for l = max(j-floor(rmin),1):min(j+floor(rmin),nely)
            fac = rmin-sqrt((i-k)^2+(j-l)^2);
            sum = sum+max(0,fac); 
            #println(fac);println(x[l,k]);println(dc[l,k]);
            dcn[j,i] += max(0,fac)*x[l,k]*dc[l,k];
          end
        end
        dcn[j,i] = dcn[j,i]/(x[j,i]*sum);
      end
    end
    #println(dcn)
    return dcn
end

check (generic function with 1 method)

In [76]:
x = top(100,60,0.5,3,2);
using Images, ImageView, TestImages
ImageView.imshow(x)

change = 0.2
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.2
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.20000000000000007
change = 0.19999999999999996
change = 0.17720034314087685
change = 0.1484527313234414
change = 0.13269882745613748
change = 0.12129119670340138
change = 0.12742764095830073
change = 0.09700495486556515
change = 0.08489979341910087
change = 0.07150220498205118
change = 0.06782950296000012
change = 0.0661438977039095
change = 0.059769866717107156
change = 0.0576676028351093
change = 0.05559775862076766
change = 0.0551196695629349
change = 0.052113683375049846
change = 0.047102754256705626
change = 0.04909597699988022
change = 0.0447536

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>235: "map(clim-mapped image, inpu…
  "annotations" => 201: "input-62" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 200: "CLim" = CLim{Float64}(0.001, 1.0) CLim{Float64} 